In [1]:
# | default_exp llm/qna

In [2]:
#| exporti
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
# from langchain.callbacks.base import BaseCallbackHandler
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory


In [3]:

# #| export

# class StreamlitStreamHandler(BaseCallbackHandler):
    
#     def __init__(self, container, initial_text=""):
#         self.container = container # streamlit container
#         self.text = initial_text

#     def on_llm_new_token(self, token: str, **kwargs) -> None:
#         self.text += token
#         self.container.write_stream(self.text)

In [4]:
#| exporti

llm = ChatOpenAI(
    openai_api_base="http://172.20.2.239:1234/v1/",
    openai_api_key="not-needed",
    model= "local_model",
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [5]:
# | export  

def get_llm_response(
    user_content,
    system_prompt="You are a QandA Bot",
    history=None,
) ->AIMessage:

    messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content=user_content),
    ]

    return llm.stream(
        messages,
        temperature=0.7,
        stream=True,
    )

In [6]:
get_llm_response("introduce yourself")

<generator object BaseChatModel.stream at 0x7f3500551fe0>

In [7]:
#+| hide
import nbdev
nbdev.nbdev_export()

In [8]:
# !jupyter nbconvert --to python llm_qna.ipynb --output ./_test/llm.py